In [40]:
from src.helper import load_matrix
import numpy as np
import pandas as pd

In [101]:
BASE_PATH = 'D:/Coding/Project/chainuser'

In [129]:
vaksinasi_df = pd.read_pickle(f'{BASE_PATH}/data/vaksinasi_simplified.pkl')
user_df = pd.read_pickle(f'{BASE_PATH}/data/process.pkl')
user_df

,indice,processed_at
007koteka,0,NaN
00apr_,1,94005.0
00a_nhs,2,27512.0
00Cikupa,3,NaN
011111pah,4,4532.0
...,...,...
_____eunoia,38784,56738.0
_____heartofmay,38785,NaN
_____onmyway,38786,NaN
______apek,38787,30074.0


In [160]:
user_ser = pd.Series(user_df.index, index=user_df['indice'], name='name')
user_ser.index.set_names(None, inplace=True)
user_ser

0              007koteka
1                 00apr_
2                00a_nhs
3               00Cikupa
4              011111pah
              ...       
38784        _____eunoia
38785    _____heartofmay
38786       _____onmyway
38787         ______apek
38788      _________elll
Name: name, Length: 38789, dtype: object

In [103]:
# mat = load_matrix('data/chainmatrix.npz').tocsr()
lil = load_matrix(f'{BASE_PATH}\data\chainmatrix.npz')
mat = lil.tocsr()

In [33]:
rows, cols = lil.nonzero()
row_ct = np.bincount(rows)
row_indices = row_ct.argsort()[::-1][:10]
# lil[row_indices].tocsc()
# row_ct[row_indices]
row_indices

array([ 7399, 32704, 21347, 21088, 16547, 11103,  9026, 26731, 34627,
       12890], dtype=int64)

In [126]:
csc = lil.tocsc()

In [183]:
main_df = pd.DataFrame({'follower': pd.Series([], dtype='str'),
                        'followee': pd.Series([], dtype='str'),
                        'delta': pd.Series([], dtype=np.float32)})
for r_in in row_indices:
    idx = cols[rows == r_in]
    x = csc[r_in, idx].toarray().flatten()
    following = user_ser.loc[idx]
    df = pd.DataFrame(np.array((x, following)).T, columns=['delta', 'followee'])
    df.sort_values('delta', inplace=True)
    df['follower'] = user_ser[r_in]
    main_df = main_df.append(df[['follower', 'followee', 'delta']], ignore_index=True)

main_df

,follower,followee,delta
0,Bunga01_Seroja,Fayola_88,449.0
1,Bunga01_Seroja,simamora_lamhot,841.0
2,Bunga01_Seroja,GuritTaman,1467.0
3,Bunga01_Seroja,dewipoe1128,1825.0
4,Bunga01_Seroja,RelawanNusanta1,2255.0
...,...,...,...
15052,Flamboyan59,pitradhana,853006.0
15053,Flamboyan59,mbahstp_,853070.0
15054,Flamboyan59,blesen3,853574.0
15055,Flamboyan59,JemuneG,854142.0


In [ ]:
main_df.to_csv(f'{BASE_PATH}/data/top10.csv', index=None)

In [ ]:
for i, j in zip(*mat.nonzero()):
    print(lil[i, j])

In [91]:
arr = mat[18141].toarray()[0]
arr.argsort()[::-1]

array([27202,  5997, 19813, ..., 25855, 25854,     0], dtype=int64)